In [1]:
from tank_kills_v3 import TankKills
from collections import deque
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import numpy as np
import json
import os
import pygame

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/dqn_tank_kills')

pygame 2.2.0 (SDL 2.0.22, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


2023-03-10 14:48:21.197697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 14:48:21.977270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-10 14:48:21.977352: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-10 14:48:21.977362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class VfApproxModel(nn.Module):
    """Neural Network for Value Function Approximation\n
    Contains Three layers 4->10->10->4
    """
    def __init__(self):
        super().__init__()
        # Input will be [player_x,player_y,enemy_x,enemy_y]
        # NOTE: Later on we will provide s,a,r,s,a
        self.layer_1 = nn.Linear(in_features=4,out_features=10)
        self.layer_2 = nn.Linear(in_features=10,out_features=10)
        self.layer_3 = nn.Linear(in_features=10,out_features=4) # 4 actions as output
        self.relu = nn.ReLU()
    
    def forward(self,features):
        out = self.relu(self.layer_1(features))
        out = self.relu(self.layer_2(out))
        out = self.layer_3(out)
        # using softmax as action-selection policy
        # out = torch.softmax(out,-1)
        return out

value_function = VfApproxModel()
value_function.to(device)

VfApproxModel(
  (layer_1): Linear(in_features=4, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=4, bias=True)
  (relu): ReLU()
)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/Q-target.jpg" style="height:400px;width:50%;float:left;">

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/sampling-training.jpg" style="height:400px;width:50%;float:right;">

In [ ]:
class Agent:
    """Main Agent Class Contiaining replay Memory and all learning params"""
    def __init__(self,replay_length,learning_rate,epsilon,max_epsilon,min_epsilon,epsilon_decay,gamma,action_size,value_function):
        self.replay_memory = deque(maxlen=replay_length)
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.action_size = action_size
        self.value_function = value_function
        self.loss_fn = nn.SmoothL1Loss()
        self.opt  = torch.optim.AdamW(value_function.parameters(), lr=learning_rate, amsgrad=True)
        self.epsilon_list = []
        self.losses =  []

    def add_experience(self,new_state,reward,running,state,action):
        """
            Adds Experience into replay_memory
            new_state = [new_player_x,new_player_y,new_enemy_x,new_enemy_y]
            state = [player_x,player_y,enemy_x,enemy_y]
            new_state ans state both are torch tensors
        """
        self.replay_memory.append((new_state,reward,running,state,action))
    
    
    def action(self,state):
        """For Taking action using e-greedy"""

        if np.random.rand() > self.epsilon:
            return np.random.randint(0,3)
        out = self.value_function(state)
        out = out.cpu().detach().numpy()
        return np.argmax(out)
    
    
    def greedy_action(self,state):
        """Predicts a action greedily only using value function"""

        out = self.value_function(state)
        out = out.cpu().detach().numpy()
        return np.argmax(out)
    
    
    def replay(self,batch_size,episode):
        """Learning From Experience Replay"""
        
        # batch size is how many batches you want to iterate to learn over
        batch = random.sample(self.replay_memory,batch_size)
        # NOTE: new_state and state should be torch tensor 
        for new_state,reward,running,state,action in batch:
            target = reward
            
            if running:
                
                target = self.value_function(new_state)
                target = target.cpu().detach().numpy()
                target = reward + self.gamma * np.amax(target)
                target = torch.tensor(target,dtype=torch.float32,device=device,requires_grad=True)

                former_target = self.value_function(state)
                former_target = former_target.cpu().detach().numpy()
                former_target = np.amax(former_target)
                former_target = torch.tensor(former_target,dtype=torch.float32,device=device,requires_grad=True)

                loss = self.loss_fn(target,former_target)
                
                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
            
            self.epsilon_list.append(self.epsilon)

    def save_weights(self,path):
        """Save Current State weights"""
        torch.save(self.value_function.state_dict(),path)


In [ ]:
all_actions = ["up","right","down","left"]
num_episodes = 2

learning_rate = 0.0001 # alpha
discount_factor = 0.90 # gamma

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.001
exploration_decay_rate = 0.0007

replay_length = 30000 # Replay memory (D) or length (N)
batch_size = 2 # Batch size to train on replay buffer

In [ ]:
agent = Agent(
    replay_length=replay_length,
    learning_rate=learning_rate,
    epsilon=exploration_rate,
    max_epsilon=max_exploration_rate,
    min_epsilon=min_exploration_rate,
    epsilon_decay=exploration_decay_rate,
    gamma=discount_factor,
    action_size=4,
    value_function=value_function
)

In [ ]:
for episode in range(num_episodes):
    env = TankKills(600,600)
    state = [300,400,0,0]
    state = torch.tensor(state,dtype=torch.float32,device=device)
    running = True
    loss_list = []
    targets_list = []
    preds_list = []
    
    while running:
        torch.cuda.empty_cache()
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > agent.epsilon:
            action = agent.greedy_action(state)
        else:
            action = random.randint(0,3)
        
        running,reward,score,pp,ep = env.play(action=all_actions[action])

        new_state = [pp[0],pp[1],ep[0],ep[1]]
        new_state = torch.tensor(new_state,dtype=torch.float32,device=device)
        # print(new_state)
        target = agent.value_function(new_state)
        # print(target)
        target = target.cpu().detach().numpy()
        target = reward + agent.gamma * np.amax(target)
        target = torch.tensor(target,dtype=torch.float32,device=device,requires_grad=True)

        former_target = agent.value_function(state)
        former_target = former_target.cpu().detach().numpy()
        former_target = np.amax(former_target)
        former_target = torch.tensor(former_target,dtype=torch.float32,device=device,requires_grad=True)

        # print(target,former_target)
        loss = agent.loss_fn(target,former_target)
        print(loss)
        
        loss.backward()
        agent.opt.step()
        loss_list.append(loss.item())
        agent.opt.zero_grad()

        
        agent.epsilon = (agent.max_epsilon-agent.min_epsilon) * np.exp(-agent.epsilon_decay*episode) + agent.min_epsilon
        agent.add_experience(new_state,reward,running,state,action)
        state = new_state

    # writer.add_scalar('Score',score,episode)
    # writer.add_scalar('Rewards',reward,episode)
    # writer.add_scalar('No. of actions',action_taken_in_episode,episode)
    # writer.add_histogram('Actions Taken',np.array(actions_taken_in_episode))
    # writer.close()
        
        
    if len(agent.replay_memory) > batch_size:
        print("Training on replay..")
        agent.replay(batch_size,episode)

pygame.display.quit() 

In [ ]:
agent.save_weights("weights_1.pth")

In [ ]:
plt.plot(loss_list)
plt.show()

In [ ]:
pygame.display.quit() 